In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

In [3]:
# データ保存ディレクトリへ移動
ls = os.getcwd().split('\\')
ls[-1] = 'data'
lsr=""
for i in ls:
    lsr += i+'\\'

# ディレクトリ移動
os.chdir(lsr)

In [4]:
train = pd.read_csv('./datasets_nb001/nb001_train.csv', index_col=0)
test = pd.read_csv('./datasets_nb001/nb001_test.csv', index_col=0)

In [5]:
print(train.shape)
print(test.shape)

(891, 20)
(418, 13)


In [6]:
# 提出データ用に受け皿を作成
PassengerID_df = test.PassengerId

In [7]:
# Sex と Embarked の One-Hot Encoding
train = pd.get_dummies(train, columns=['Sex','Embarked'])
test = pd.get_dummies(test, columns=['Sex','Embarked'])

# 不要な列の削除
train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'CategoricalAge', 'SibSp_0_1_2over',
            'Parch_0_1_2_3over', 'FamilySize', 'IsAlone', 'CategoricalFare',
            'Title', 'Title_num'],
            axis=1, inplace=True)

test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Title', 'Title_num'],
            axis=1, inplace=True)

In [8]:
X_train = train.drop(['Survived'], axis=1)
y_train = train['Survived']

In [9]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# X_train と y_train を train と valid に分割
train_x, valid_x, train_y, valid_y = train_test_split(X_train, y_train,
                                                      test_size=0.33, random_state=0)

# K分割交差検証

In [10]:
# LightGBMの分類器をインスタンス化
gbm = lgb.LGBMClassifier(objective='binary')

# 学習
gbm.fit(train_x, train_y, eval_set=[(train_x,train_y),(valid_x,valid_y)],
        callbacks=[lgb.early_stopping(stopping_rounds=20,
                                      verbose=True)]
        )

[LightGBM] [Info] Number of positive: 231, number of negative: 365
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000229 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 189
[LightGBM] [Info] Number of data points in the train set: 596, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.387584 -> initscore=-0.457480
[LightGBM] [Info] Start training from score -0.457480
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 20 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[L

c:\Users\nao23\Documents\Python_Projects\Kaggle_Titanic_Machine_Learning_from_Disaster_311\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
found 0 physical cores < 1
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "c:\Users\nao23\Documents\Python_Projects\Kaggle_Titanic_Machine_Learning_from_Disaster_311\Lib\site-packages\joblib\externals\loky\backend\context.py", line 282, in _count_physical_cores
    raise ValueError(f"found {cpu_count_physical} physical cores < 1")


LGBMClassifier(objective='binary')

In [11]:
# valid_xについて推論
oof = gbm.predict(valid_x, num_iteration=gbm.best_iteration_)
print('score', round(accuracy_score(valid_y, oof)*100,2))

score 82.37
